In [1]:
import os
from datetime import datetime
import time
import numpy as np
import h5py
import random
from PIL import Image
import tensorflow as tf
import json
from model import Model
print("haha")

/Users/wenjiechen/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/wenjiechen/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/wenjiechen/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/wenjiechen/opt/anaconda3/lib/python3.7/site-pack

haha


In [2]:
def build_batch(path_to_tfrecords_file, num_examples, batch_size, shuffled):
    assert tf.gfile.Exists(path_to_tfrecords_file), '%s not found' % path_to_tfrecords_file

    filename_queue = tf.train.string_input_producer([path_to_tfrecords_file], num_epochs=None)
    #image, length, digits = read_and_decode(filename_queue)

    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(
            serialized_example,
            features={
                'image': tf.FixedLenFeature([], tf.string),
                'length': tf.FixedLenFeature([], tf.int64),
                'digits': tf.FixedLenFeature([5], tf.int64)
            })

    image = tf.decode_raw(features['image'], tf.uint8)
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    image = tf.multiply(tf.subtract(image, 0.5), 2)
    image = tf.reshape(image, [64, 64, 3])
    image = tf.random_crop(image, [54, 54, 3])
    
    length = tf.cast(features['length'], tf.int32)
    digits = tf.cast(features['digits'], tf.int32)
    
    
    
    min_queue_examples = int(0.4 * num_examples)

    image_batch, length_batch, digits_batch = tf.train.batch([image, length, digits],
                                                                     batch_size=batch_size,
                                                                     num_threads=2,
                                                                     capacity=min_queue_examples + 3 * batch_size)
    return image_batch, length_batch, digits_batch

In [3]:
def write_summary(path_to_eval_log_dir):
    summary_writer = tf.summary.FileWriter(path_to_eval_log_dir)
    return summary_writer

def evaluate(writer, path_to_checkpoint, path_to_tfrecords_file, num_examples, global_step):
    batch_size = 128
    num_batches = num_examples // batch_size
    needs_include_length = False

    with tf.Graph().as_default():
        image_batch, length_batch, digits_batch = build_batch(path_to_tfrecords_file,
                                                                         num_examples=num_examples,
                                                                         batch_size=batch_size,
                                                                         shuffled=False)
        length_logits, digits_logits = Model.inference(image_batch, drop_rate=0.0)
        length_predictions = tf.argmax(length_logits, axis=1)
        digits_predictions = tf.argmax(digits_logits, axis=2)


        labels = digits_batch
        predictions = digits_predictions

        labels_string = tf.reduce_join(tf.as_string(labels), axis=1)
        predictions_string = tf.reduce_join(tf.as_string(predictions), axis=1)

        accuracy, update_accuracy = tf.metrics.accuracy(
        labels=labels_string,
        predictions=predictions_string
            )

        tf.summary.image('image', image_batch)
        tf.summary.scalar('accuracy', accuracy)
        tf.summary.histogram('variables',
                                 tf.concat([tf.reshape(var, [-1]) for var in tf.trainable_variables()], axis=0))
        summary = tf.summary.merge_all()

        with tf.Session() as sess:
            sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])
            coord = tf.train.Coordinator()
            threads = tf.train.start_queue_runners(sess=sess, coord=coord)

            restorer = tf.train.Saver()
            restorer.restore(sess, path_to_checkpoint)

            for _ in range(num_batches):
                sess.run(update_accuracy)

            accuracy_val, summary_val = sess.run([accuracy, summary])
            writer.add_summary(summary_val, global_step=global_step)

            coord.request_stop()
            coord.join(threads)

    return accuracy_val




In [5]:
# Build the training process

def train(path_to_train_tfrecords_file, num_train_examples, path_to_val_tfrecords_file, num_val_examples,
           path_to_train_log_dir, training_options, history_file_path):
    batch_size = training_options['batch_size']
    initial_patience = training_options['patience']
    num_steps_to_show_loss = 10
    num_steps_to_check = 100

    with tf.Graph().as_default():
        image_batch, length_batch, digits_batch = build_batch(path_to_train_tfrecords_file,
                                                                     num_examples=num_train_examples,
                                                                     batch_size=batch_size,
                                                                     shuffled=True)
        
        length_logtis, digits_logits = Model.inference(image_batch, drop_rate=0.2)
        loss = Model.loss(length_logtis, digits_logits, length_batch, digits_batch)
        
        global_step = tf.Variable(0, name='global_step', trainable=False)
        learning_rate = tf.train.exponential_decay(training_options['learning_rate'], global_step=global_step,
                                                   decay_steps=training_options['decay_steps'], decay_rate=training_options['decay_rate'], staircase=True)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        train_op = optimizer.minimize(loss, global_step=global_step)

        tf.summary.image('image', image_batch)
        tf.summary.scalar('loss', loss)
        tf.summary.scalar('learning_rate', learning_rate)
        summary = tf.summary.merge_all()

        with tf.Session() as sess:
            summary_writer = tf.summary.FileWriter(path_to_train_log_dir, sess.graph)
            evaluator = write_summary(os.path.join(path_to_train_log_dir, 'eval/val'))
            #evaluator = Evaluator(os.path.join(path_to_train_log_dir, 'eval/val'))
    
            #tf.global_variables_initializer()
            sess.run(tf.global_variables_initializer())
            coord = tf.train.Coordinator()
            threads = tf.train.start_queue_runners(sess=sess, coord=coord)

            saver = tf.train.Saver()

            print ('Start training')
            patience = initial_patience
            best_accuracy = 0.0
            duration = 0.0
            epoch = 32000
            k=0
            accuracy_history=[]
            for i in range(32000):
                k+=1
                start_time = time.time()
                _, loss_val, summary_val, global_step_val, learning_rate_val = sess.run([train_op, loss, summary, global_step, learning_rate])
                duration += time.time() - start_time

                if global_step_val % num_steps_to_show_loss == 0:
                    examples_per_sec = batch_size * num_steps_to_show_loss / duration
                    duration = 0.0
                    print ('%s: step %d, loss = %f ' % (
                        datetime.now(), global_step_val, loss_val))

                if global_step_val % num_steps_to_check != 0:
                    continue

                summary_writer.add_summary(summary_val, global_step=global_step_val)


                path_to_latest_checkpoint_file = saver.save(sess, os.path.join(path_to_train_log_dir, 'latest.ckpt'))
                accuracy = evaluate(evaluator, path_to_latest_checkpoint_file, path_to_val_tfrecords_file,
                                              num_val_examples,
                                              global_step_val)
                print ('Validation accuracy is= %f, best accuracy %f' % (accuracy, best_accuracy))
                if k%10==0:
                    accuracy_history.append(accuracy)
                if accuracy > best_accuracy:
                    path_to_checkpoint_file = saver.save(sess, os.path.join(path_to_train_log_dir, 'model.ckpt'),
                                                         global_step=global_step_val)
                    print ('Save file to: %s' % path_to_checkpoint_file)
                    patience = initial_patience
                    best_accuracy = accuracy
                else:
                    patience -= 1


                if k==32000:
                    break

            coord.request_stop()
            coord.join(threads)
            with open(history_file_path, 'w', encoding='utf-8') as f:
                f.write('times, acuuracy\n')
                for i,d in enumerate(accuracy_history):
                    f.write(str(i) + ',' + str(d) + '\n')
            print ('Training progess is finished')

In [8]:
# First
#Train the model 
#define the folder path
train_tfrecords_file = 'data/train.tfrecords'
val_tfrecords_file = 'data/val.tfrecords'
tfrecords_meta_file = 'data/meta.json'
log_dir = 'logs_hyper/train'
history_file_path = 'data/history_hyper2_1.csv' #history file
opt = {
    'batch_size': 32, #batch_size=16,32,64
    'learning_rate': 1e-2, #learning_rate = 1e-2, 5e-3, 1e-3
    'patience': 1000,
    'decay_steps': 10000,
    'decay_rate': 0.9
    }
with open(tfrecords_meta_file, 'r') as f:
    content = json.load(f)
    num_train_examples = content['num_examples']['train']
    num_val_examples = content['num_examples']['val']
    num_test_examples = content['num_examples']['test']
    
    
#train the model     
train(train_tfrecords_file, num_train_examples,
           val_tfrecords_file, num_val_examples,
           log_dir, 
           opt, history_file_path)

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.TFRecor

In [6]:
# Second
#Train the model 
#define the folder path
train_tfrecords_file = 'data/train.tfrecords'
val_tfrecords_file = 'data/val.tfrecords'
tfrecords_meta_file = 'data/meta.json'
log_dir = 'logs_hyper/train'
history_file_path = 'data/history_hyper2_2.csv' #history file
opt = {
    'batch_size': 32, #batch_size=16,32,64
    'learning_rate': 5e-3, #learning_rate = 1e-2, 5e-3, 1e-3
    'patience': 1000,
    'decay_steps': 10000,
    'decay_rate': 0.9
    }
with open(tfrecords_meta_file, 'r') as f:
    content = json.load(f)
    num_train_examples = content['num_examples']['train']
    num_val_examples = content['num_examples']['val']
    num_test_examples = content['num_examples']['test']
    
    
#train the model     
train(train_tfrecords_file, num_train_examples,
           val_tfrecords_file, num_val_examples,
           log_dir, 
           opt, history_file_path)

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.TFRecor

In [7]:
# Third
#Train the model 
#define the folder path
train_tfrecords_file = 'data/train.tfrecords'
val_tfrecords_file = 'data/val.tfrecords'
tfrecords_meta_file = 'data/meta.json'
log_dir = 'logs_hyper/train'
history_file_path = 'data/history_hyper2_3.csv' #history file
opt = {
    'batch_size': 32, #batch_size=16,32,64
    'learning_rate': 1e-3, #learning_rate = 1e-2, 5e-3, 1e-3
    'patience': 1000,
    'decay_steps': 10000,
    'decay_rate': 0.9
    }
with open(tfrecords_meta_file, 'r') as f:
    content = json.load(f)
    num_train_examples = content['num_examples']['train']
    num_val_examples = content['num_examples']['val']
    num_test_examples = content['num_examples']['test']
    
    
#train the model     
train(train_tfrecords_file, num_train_examples,
           val_tfrecords_file, num_val_examples,
           log_dir, 
           opt, history_file_path)

Start training
2019-12-08 19:21:38.007842: step 10, loss = 13.879457 
2019-12-08 19:21:38.396699: step 20, loss = 13.811701 
2019-12-08 19:21:38.745621: step 30, loss = 13.742438 
2019-12-08 19:21:39.086396: step 40, loss = 13.667187 
2019-12-08 19:21:39.437931: step 50, loss = 13.593492 
2019-12-08 19:21:39.789404: step 60, loss = 13.506971 
2019-12-08 19:21:40.147574: step 70, loss = 13.307712 
2019-12-08 19:21:40.496621: step 80, loss = 13.122448 
2019-12-08 19:21:40.842886: step 90, loss = 12.756750 
2019-12-08 19:21:41.195325: step 100, loss = 11.482825 
INFO:tensorflow:Restoring parameters from logs_hyper/train/latest.ckpt
Validation accuracy is= 0.020625, best accuracy 0.000000
Save file to: logs_hyper/train/model.ckpt-100
2019-12-08 19:21:49.341416: step 110, loss = 9.501649 
2019-12-08 19:21:49.602588: step 120, loss = 7.604829 
2019-12-08 19:21:49.840330: step 130, loss = 7.104119 
2019-12-08 19:21:50.082495: step 140, loss = 6.639066 
2019-12-08 19:21:50.333192: step 150, lo

In [5]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add()
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>" % size
    return strip_def


def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script src="//cdnjs.cloudflare.com/ajax/libs/polymer/0.3.3/platform.js"></script>
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph' + str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1000px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [6]:
with tf.Session() as sess:
    saver = tf.train.import_meta_graph('logs_hyper/train/latest.ckpt.meta')
    graph = tf.get_default_graph()
    show_graph(graph)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
